<a href="https://colab.research.google.com/github/arminabdeh/Localization3D/blob/smlm/optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

In [ ]:
# @markdown **Step 0** Prepare Colab instance (~5 mins)
# @markdown  1. install __luenn__ package
# @markdown  2. install __decode__ package
# @markdown  3. install __torchvision__
# @markdown  4. install __torch 1.12.0__
# @markdown  5. install __optuna__ and __optuna-dashboard__
# @markdown  3. install __torachtext__ and __torchaudio__
%%capture
!pip install Optuna
!pip install Optuna-dashboard
!pip uninstall -y torch torchvision torchaudio torchtext
!pip install torch==1.12.0
import torch
import yaml
!rm -r sample_data/*
!wget -c https://raw.githubusercontent.com/arminabdeh/Localization3D/smlm/gateway.yaml
with open('gateway.yaml') as f:
  gateway = yaml.safe_load(f)


wheel_spline = gateway['wheels']['spline_py310']
wheel_spline_name = "spline-0.11.1.dev0-cp310-cp310-linux_x86_64.whl"
!wget -O $wheel_spline_name $wheel_spline
!pip install $wheel_spline_name

wheel_decode = gateway['wheels']['decode']
!pip install $wheel_decode

wheel_luenn =  gateway['wheels']['luenn']
!pip install $wheel_luenn

In [2]:
#@markdown **Step 3** Mount Google Drive
#@markdown
#@markdown Execute this cell to connect your Google Drive
#@markdown This is important to save your trained model. An alternative would be to download it manually, this is however not recommended.


#@markdown * Click on the URL.

#@markdown * Sign in your Google Account.

#@markdown * Copy and Enter the authorization code (if it asked for) or allow access.

#@markdown * Click on "Files" site on the right. Refresh the site. Your Google Drive folder should now be available here as "drive".

#@markdown Your Google Drive is accessible in Colab as `gdrive/My Drive/[YOUR Folder / File]`. **Important** It might take a while until files are being synced (particularly large files).
# mount user's Google Drive to Google Colab.
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
from luenn.model import UNet
from luenn.generic import fly_simulator
from luenn.localization import localizer_machine
from luenn.evaluate import reg_classification

import decode.utils.param_io as param_io

from torch.utils.data import Dataset, TensorDataset,DataLoader
import torch.nn as nn
from torchsummary import summary
import torch
import optuna
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import threading
from google.colab import output
from optuna_dashboard import run_server
from optuna.trial import TrialState

epochs = 20 # @param {type:"integer"}
ds_size = 64 # @param {type:"integer"}
batch_size = 16 # @param {type:"integer"}
emitter_av = 8 # @param {type:"raw"}
lr_lim_down = 1e-6 # @param {type:"raw"}
lr_lim_up = 1e-3 # @param {type:"raw"}
step_min = 1 # @param {type:"integer"}
step_max = 20 # @param {type:"integer"}
gamma_min = 0.1 # @param {type:"raw"}
gamma_max = 0.99 # @param {type:"raw"}
param_path = "/content/gdrive/MyDrive/optuna/param/param.yaml"
param = param_io.load_params(param_path)
calib_path = "/content/gdrive/MyDrive/optuna/calib/spline_calibration_3d_as_3dcal.mat"
param.InOut.calibration_file = calib_path
param.Simulation.emitter_av = emitter_av
param.HyperParameter.pseudo_ds_size = ds_size
param.TestSet.test_size = 1


def define_model(trial):
	input_channels = 1
	initializer_choices = ['kaiming_uniform_', 'xavier_uniform_', 'xavier_normal_', 'kaiming_normal_']
	initializer_str = trial.suggest_categorical('initializer', initializer_choices)
	if initializer_str == 'kaiming_uniform_':
		initializer = nn.init.kaiming_uniform_
	elif initializer_str == 'xavier_uniform_':
		initializer = nn.init.xavier_uniform_
	elif initializer_str == 'xavier_normal_':
		initializer = nn.init.xavier_normal_
	elif initializer_str == 'kaiming_normal_':
		initializer = nn.init.kaiming_normal_

	activation_choices = ['ReLU', 'ELU', 'GELU']
	activation = trial.suggest_categorical('activation', activation_choices)
	if activation == 'ReLU':
		activation_func = nn.ReLU
	elif activation == 'ELU':
		activation_func = nn.ELU
	elif activation == 'GELU':
		activation_func = nn.GELU

	pred_channels = trial.suggest_categorical('pred_channels', [16, 32, 64])
	kernel_size_encoder = trial.suggest_categorical('kernel_size_encoder', [3,5])
	kernel_size_decoder = trial.suggest_categorical('kernel_size_decoder', [3,5])
	kernel_size_output = trial.suggest_categorical('kernel_size_output', [3,5])

	model = UNet(initializer, activation_func, input_channels, pred_channels, kernel_size_encoder, kernel_size_decoder, kernel_size_output)
	return model

def train_one_epoch(model,optimizer,criterion,dataloader_train):
	model.train()
	steps = len(dataloader_train)
	# tqdm_enum = tqdm(total=steps, smoothing=0.)
	train_loss = 0
	for data in dataloader_train:
		inputs, labels = data
		inputs = inputs.cuda()
		labels = labels.cuda()
		optimizer.zero_grad()
		outputs = model(inputs)
		loss = criterion(outputs, labels)
		train_loss+=loss.item()
		loss.backward()
		optimizer.step()
		# tqdm_enum.update(1)
		inputs = inputs.cpu()
		labels = labels.cpu()
		torch.cuda.empty_cache()
	train_loss /= len(dataloader_train)
	return model, train_loss

def dataloader(sim):
	x_train, y_train, gt_train = sim.ds_train()
	dataset_train = torch.utils.data.TensorDataset(x_train, y_train)
	return dataset_train

def objective(trial):
	model = define_model(trial).to('cuda')
	optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "AdamW", "RMSprop", "SGD"])
	# optimizer_name = 'Adam'
	lr = trial.suggest_float("lr", lr_lim_down, lr_lim_up, log=True)
	# lr = 0.0001
	optimizer = getattr(torch.optim, optimizer_name)(model.parameters(), lr=lr)
	step = trial.suggest_int("step", 1, 20, log=True)
	gamma = trial.suggest_float("gamma", 0.1, 1, log=True)

	criterion = torch.nn.MSELoss()
	simulator = fly_simulator(param,report=False)
	lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step, gamma=gamma)
	# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=.9)
	tqdm_enum = tqdm(total=epochs, smoothing=0.)
	for epoch in range(epochs):
		train_set = dataloader(simulator)
		dataloader_train  = DataLoader(train_set, batch_size=batch_size,num_workers=0,shuffle=False, pin_memory=False)
		model, train_loss = train_one_epoch(model,optimizer,criterion,dataloader_train)
		lr_scheduler.step()
		tqdm_enum.update(1)
		trial.report(train_loss, epoch)
		if trial.should_prune():
			raise optuna.exceptions.TrialPruned()
	return train_loss

# if __name__ == "__main__":
storage = optuna.storages.InMemoryStorage()
study = optuna.create_study(storage=storage)
study.optimize(objective, n_trials=20)
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))
# # Start Optuna Dashboard
port = 8081
thread = threading.Thread(target=run_server, args=(storage,), kwargs={"port": port})
thread.start()
output.serve_kernel_port_as_window(port, path='/dashboard/')
# run_server(storage)

[I 2023-12-03 19:23:42,762] A new study created in memory with name: no-name-867e5d73-f545-48df-9fd0-5afd4e15b342
127.0.0.1 - - [03/Dec/2023 19:23:43] "GET /api/studies/0?after=20 HTTP/1.1" 200 8032


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2023-12-03 19:24:43,359] Trial 0 finished with value: 47345.5890625 and parameters: {'initializer': 'kaiming_uniform_', 'activation': 'ELU', 'pred_channels': 32, 'kernel_size_encoder': 5, 'kernel_size_decoder': 5, 'kernel_size_output': 5, 'optimizer': 'RMSprop', 'lr': 1.660600490192287e-05, 'step': 1, 'gamma': 0.10890156739113718}. Best is trial 0 with value: 47345.5890625.


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2023-12-03 19:25:49,223] Trial 1 finished with value: 45330.13125 and parameters: {'initializer': 'kaiming_uniform_', 'activation': 'ELU', 'pred_channels': 64, 'kernel_size_encoder': 5, 'kernel_size_decoder': 3, 'kernel_size_output': 3, 'optimizer': 'SGD', 'lr': 1.4069033426079659e-05, 'step': 1, 'gamma': 0.24524594464454413}. Best is trial 1 with value: 45330.13125.


...

[I 2023-12-03 19:32:41,462] Trial 19 pruned. 


Study statistics: 
  Number of finished trials:  20
  Number of pruned trials:  10
  Number of complete trials:  10
Best trial:
  Value:  42969.196484375
  Params: 
    initializer: xavier_uniform_
    activation: ELU
    pred_channels: 16
    kernel_size_encoder: 3
    kernel_size_decoder: 3
    kernel_size_output: 5
    optimizer: RMSprop
    lr: 0.0003665133010189843
    step: 11
    gamma: 0.439517338770377


Bottle v0.12.25 server starting up (using WSGIRefServer())...
Listening on http://localhost:8081/
Hit Ctrl-C to quit.

Exception in thread Thread-13 (run_server):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/optuna_dashboard/_app.py", line 497, in run_server
    run(app, host=host, port=port)
  File "/usr/local/lib/python3.10/dist-packages/bottle.py", line 3175, in run


<IPython.core.display.Javascript object>

    server.run(app)
  File "/usr/local/lib/python3.10/dist-packages/bottle.py", line 2801, in run
    srv = make_server(self.host, self.port, app, server_cls, handler_cls)
  File "/usr/lib/python3.10/wsgiref/simple_server.py", line 154, in make_server
    server = server_class((host, port), handler_class)
  File "/usr/lib/python3.10/socketserver.py", line 452, in __init__
    self.server_bind()
  File "/usr/lib/python3.10/wsgiref/simple_server.py", line 50, in server_bind
    HTTPServer.server_bind(self)
  File "/usr/lib/python3.10/http/server.py", line 137, in server_bind
    socketserver.TCPServer.server_bind(self)
  File "/usr/lib/python3.10/socketserver.py", line 466, in server_bind
    self.socket.bind(self.server_address)
OSError: [Errno 98] Address already in use


In [3]:
# @title import libraries
from luenn.model import UNet
from luenn.generic import fly_simulator
from luenn.localization import localizer_machine
from luenn.evaluate import reg_classification

import decode.utils.param_io as param_io

from torch.utils.data import Dataset, TensorDataset,DataLoader
import torch.nn as nn
from torchsummary import summary

import optuna
from tqdm import tqdm
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np

In [8]:
# @title Default title text
#@markdown > Set main parameters:
epochs = 4 # @param {type:"integer"}
ds_size = 199 # @param {type:"integer"}
batch_size = 4 # @param {type:"integer"}
emitter_av = 10 # @param {type:"raw"}
lr_lim_down = 1e-6 # @param {type:"raw"}
lr_lim_up = 1e-3 # @param {type:"raw"}
step_min = 1 # @param {type:"integer"}
step_max = 20 # @param {type:"integer"}
gamma_min = 0.1 # @param {type:"raw"}
gamma_max = 0.99 # @param {type:"raw"}
param_path = "/content/gdrive/MyDrive/optuna/param/param.yaml"
param = param_io.load_params(param_path)
calib_path = "/content/gdrive/MyDrive/optuna/calib/spline_calibration_3d_as_3dcal.mat"
param.InOut.calibration_file = calib_path
param.Simulation.emitter_av = emitter_av
param.HyperParameter.pseudo_ds_size = ds_size
param.TestSet.test_size = 1

def define_model(trial):
    input_channels = 1
    initializer_choices = ['kaiming_uniform_', 'xavier_uniform_', 'xavier_normal_', 'kaiming_normal_']
    initializer_str = trial.suggest_categorical('initializer', initializer_choices)
    if initializer_str == 'kaiming_uniform_':
        initializer = nn.init.kaiming_uniform_
    elif initializer_str == 'xavier_uniform_':
        initializer = nn.init.xavier_uniform_
    elif initializer_str == 'xavier_normal_':
        initializer = nn.init.xavier_normal_
    elif initializer_str == 'kaiming_normal_':
        initializer = nn.init.kaiming_normal_

    activation_choices = ['ReLU', 'ELU', 'GELU']
    activation = trial.suggest_categorical('activation', activation_choices)
    if activation == 'ReLU':
        activation_func = nn.ReLU
    elif activation == 'ELU':
        activation_func = nn.ELU
    elif activation == 'GELU':
        activation_func = nn.GELU

    pred_channels = trial.suggest_categorical('pred_channels', [32, 64, 96])
    kernel_size_encoder = trial.suggest_categorical('kernel_size_encoder', [3,5])
    kernel_size_decoder = trial.suggest_categorical('kernel_size_decoder', [3,5])
    kernel_size_output = trial.suggest_categorical('kernel_size_output', [3,5,7])



    model = UNet(initializer, activation, input_channels, pred_channels, kernel_size_encoder, kernel_size_decoder, kernel_size_output)
    return model

def train_one_epoch(model,optimizer,criterion,dataloader_train):
    model.train()
    steps = len(dataloader_train)
    # tqdm_enum = tqdm(total=steps, smoothing=0.)
    train_loss = 0
    for data in dataloader_train:
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        train_loss+=loss.item()
        loss.backward()
        optimizer.step()
        # tqdm_enum.update(1)
        inputs = inputs.cpu()
        labels = labels.cpu()
        torch.cuda.empty_cache()
    train_loss /= len(dataloader_train)
    return model, train_loss

def validation(model,criterion,dataloader_test,gt_val):
    model.eval()
    steps = len(dataloader_test)
    # tqdm_enum = tqdm(total=steps, smoothing=0.)
    val_loss = 0
    # loc_file = None
    # pr_gt = pd.DataFrame([])
    with torch.no_grad():
        for data in dataloader_test:
            inputs, labels = data
            outputs = model(inputs.cuda())
            loss = criterion(outputs, labels.cuda())
            val_loss += loss.item()
            # tqdm_enum.update(1)
            inputs = inputs.cpu()
            labels = labels.cpu()
            torch.cuda.empty_cache()
            # temp_out = np.moveaxis(outputs.cpu().numpy(), 1, -1)
            # loc_file = temp_out if loc_file is None else np.concatenate((loc_file, temp_out), axis=0)
    val_loss /=len(dataloader_test)
    return val_loss
    # pr_gt = localizer_machine(param,loc_file,gt_val,save=False).localization_3D()
    # val_ji = reg_classification(pr_gt).jaccardian_index()
    # return val_loss,val_ji

def dataloader(sim):
    # x_test, y_test, gt_test = sim.ds_test()
    # dataset_test = torch.utils.data.TensorDataset(x_test, y_test)
    x_train, y_train, gt_train = sim.ds_train()
    dataset_train = torch.utils.data.TensorDataset(x_train, y_train)
    return dataset_train

def objective(trial):
    model = define_model(trial).to('cuda')
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "AdamW", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", lr_lim_down, lr_lim_up, log=True)
    optimizer = getattr(torch.optim, optimizer_name)(model.parameters(), lr=lr)
    step = trial.suggest_int("step", 1, 20, log=True)
    gamma = trial.suggest_float("gamma", 0.1, 1, log=True)

    criterion = torch.nn.MSELoss()
    simulator = fly_simulator(param,report=False)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step, gamma=gamma)
    tqdm_enum = tqdm(total=epochs, smoothing=0.)
    for epoch in range(epochs):
        train_set = dataloader(simulator)
        dataloader_train  = DataLoader(train_set, batch_size=batch_size,num_workers=0,shuffle=False, pin_memory=False)
        model, train_loss = train_one_epoch(model,optimizer,criterion,dataloader_train)
        trial.report(train_loss, epoch)
        lr_scheduler.step()
        tqdm_enum.update(1)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    return train_loss



In [9]:
study = optuna.create_study(direction="minimize")
study.optimize(objective,n_trials=20000, timeout=1)

[I 2023-12-02 20:17:44,891] A new study created in memory with name: no-name-3633ff70-382a-443d-a0fc-23cc7d7ba862
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <function kaiming_uniform_ at 0x79e43d37eef0> which is of type function.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <function xavier_uniform_ at 0x79e43d37ed40> which is of type function.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <function xavier_normal_ at 0x79e43d37edd0> which is of type fun

  0%|          | 0/4 [00:00<?, ?it/s]

[I 2023-12-02 20:18:16,196] Trial 0 finished with value: 57414.377265625 and parameters: {'pred_channels': 32, 'kernel_size_encoder': 3, 'kernel_size_decoder': 3, 'kernel_size_output': 5, 'initializer': <function xavier_normal_ at 0x79e43d37edd0>, 'activation': <class 'torch.nn.modules.activation.ELU'>, 'optimizer': 'Adam', 'lr': 0.00033830536926917514, 'step': 1, 'gamma': 0.8134864957114372}. Best is trial 0 with value: 57414.377265625.
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <function kaiming_uniform_ at 0x79e43d37eef0> which is of type function.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <function xavier_uniform_ at 0x79e43d37ed4

  0%|          | 0/4 [00:00<?, ?it/s]

[I 2023-12-02 20:18:52,817] Trial 1 finished with value: 65743.70296875 and parameters: {'pred_channels': 32, 'kernel_size_encoder': 5, 'kernel_size_decoder': 3, 'kernel_size_output': 3, 'initializer': <function kaiming_uniform_ at 0x79e43d37eef0>, 'activation': <class 'torch.nn.modules.activation.GELU'>, 'optimizer': 'Adam', 'lr': 0.0002209553217600567, 'step': 19, 'gamma': 0.20424467826933979}. Best is trial 0 with value: 57414.377265625.


In [7]:
print(study.best_trial)
print(study.best_params)

FrozenTrial(number=1, state=TrialState.COMPLETE, values=[58836.043671875], datetime_start=datetime.datetime(2023, 12, 2, 20, 16, 0, 467973), datetime_complete=datetime.datetime(2023, 12, 2, 20, 16, 27, 8267), params={'pred_channels': 32, 'kernel_size_encoder': 3, 'kernel_size_decoder': 3, 'kernel_size_output': 3, 'initializer': <function xavier_normal_ at 0x79e43d37edd0>, 'activation': <class 'torch.nn.modules.activation.ReLU'>, 'optimizer': 'SGD', 'lr': 0.0001013547672205578, 'step': 19, 'gamma': 0.28221291043531227}, user_attrs={}, system_attrs={}, intermediate_values={0: 62142.4941796875, 1: 61175.194140625, 2: 56388.4921875, 3: 58836.043671875}, distributions={'pred_channels': CategoricalDistribution(choices=(32, 64)), 'kernel_size_encoder': CategoricalDistribution(choices=(3,)), 'kernel_size_decoder': CategoricalDistribution(choices=(3,)), 'kernel_size_output': CategoricalDistribution(choices=(3,)), 'initializer': CategoricalDistribution(choices=(<function kaiming_uniform_ at 0x79

In [4]:
!pip install optuna
!pip install optuna_dashboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 11.2 MB/s eta 0:00:00


In [7]:
import optuna
import threading
from google.colab import output
from optuna_dashboard import run_server

def objective(trial):
    x = trial.suggest_float("x", -100, 100)
    return (x - 2) ** 2

# Run optimization
storage = optuna.storages.InMemoryStorage()
study = optuna.create_study(storage=storage)
study.optimize(objective, n_trials=100)

# Start Optuna Dashboard
port = 8081
thread = threading.Thread(target=run_server, args=(storage,), kwargs={"port": port})
thread.start()
output.serve_kernel_port_as_window(port, path='/dashboard/')

[I 2023-12-03 18:43:39,703] A new study created in memory with name: no-name-a82bc960-c633-4482-bc22-a7858cf871c5
[I 2023-12-03 18:43:39,706] Trial 0 finished with value: 7494.160083045613 and parameters: {'x': 88.56881703619158}. Best is trial 0 with value: 7494.160083045613.
[I 2023-12-03 18:43:39,708] Trial 1 finished with value: 1666.6847470094858 and parameters: {'x': -38.82505048385716}. Best is trial 1 with value: 1666.6847470094858.
[I 2023-12-03 18:43:39,710] Trial 2 finished with value: 5536.736612404874 and parameters: {'x': -72.40925085232934}. Best is trial 1 with value: 1666.6847470094858.
[I 2023-12-03 18:43:39,712] Trial 3 finished with value: 1915.8359158940702 and parameters: {'x': -41.77026291780837}. Best is trial 1 with value: 1666.6847470094858.
[I 2023-12-03 18:43:39,713] Trial 4 finished with value: 95.76453111057317 and parameters: {'x': -7.785935372286758}. Best is trial 4 with value: 95.76453111057317.
[I 2023-12-03 18:43:39,715] Trial 5 finished with value: 

<IPython.core.display.Javascript object>

    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/optuna_dashboard/_app.py", line 497, in run_server
    run(app, host=host, port=port)
  File "/usr/local/lib/python3.10/dist-packages/bottle.py", line 3175, in run
    server.run(app)
  File "/usr/local/lib/python3.10/dist-packages/bottle.py", line 2801, in run
    srv = make_server(self.host, self.port, app, server_cls, handler_cls)
  File "/usr/lib/python3.10/wsgiref/simple_server.py", line 154, in make_server
    server = server_class((host, port), handler_class)
  File "/usr/lib/python3.10/socketserver.py", line 452, in __init__
    self.server_bind()
  File "/usr/lib/python3.10/wsgiref/simple_server.py", line 50, in server_bind
    HTTPServer.server_bind(self)
  File "/usr/lib/python3.10/http/server.py", line 137, in server_bind
    socketserver.TCPServer.server_bind(self)
  File "/usr/lib/python3.10/socketserver.py", line 466, in server_bind
    self.socket.bind(self.server_address)
OSEr